Topics and Trends

    Alter code example 2 (ex2_search.ipynb) to produce a list of 1000 tweets about a topic.
    How does twitter interpret a two word query like "data science"
    Eliminate retweets [hint: look at the tweet object! https://dev.twitter.com/overview/api/tweets]
    For each tweet original tweet, list the number of times you see it retweeted.
    Get a list of the URLs that are embedded in Tweets with your topic.

Geolocation from Search API This section will require you to investigate the filter function in example 2 in more detail.

    Get the last 50 tweets from West Lafayette.
    Get the last 50 tweets from Times Square.
    Using timestamps, can you estimate whether people tweet more often in West Lafayette or Times Square?
    A Premier League soccer game happened today between Liverpool and Chelsea. Using two geo searches, see if you can tell which city hosted the game. Note: if you do this some other day, you should pick a new sporting event.

Geolocation in the streaming API

    Alter the streaming algorithm to include a "locations" filter. You need to use the order sw_lng, sw_lat, ne_lng, ne_lat for the four coordinates. (Recall the stop button will stop an active process like the stream.)
    What are people tweeting about in Times Square today? (Bonus points: set up a bounding box around TS and around NYC as a whole.)
    Can you find words that are more likely to appear in Times Square (hint: you'll need two bounding boxes)?
    Purdue is playing basketball against Iowa tonight. Set up a bounding box around West Lafayette and Iowa City, Iowa. Can you identify tweets about basketball? Who tweets more about the game? Can you tell which team is the home team?

Geolocation hint: You can use d = api.search(geocode='[lng],[lat],5mi) to get Tweets from a 5 mile radius around a point. Use Google or Bing maps to get a similar bounding box around Fenway Park.


Who are my followers?

    Alter code example 1 (ex1_get_user_info.ipynb) to get your followers.
    For each of your followers, get *their* followers (investigate time.sleep to throttle your computation)
    Identify the follower you have that also follows the most of your followers.
    How many handles follow you but none of your followers?
    Repeat this for people you follow, rather than those that follow you.

In [ ]:
import tweepy
from twitter_authentication import CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

In [ ]:
api = tweepy.API(auth, wait_on_rate_limit=True)

# The easiest way to get lots of tweets is with tweepy.Cursor. 
# It keeps track of where you are, and gets a big list

for tweet in tweepy.Cursor(api.search, 'data science').items(1000):
    print(tweet.text)

In [ ]:
# Eliminating retweets
for tweet in tweepy.Cursor(api.search, 'data science').items(100):
    if 'retweeted_status' in tweet._json:
        continue
    else:
        print(tweet.text)


In [ ]:
# Counting retweets
result = {}
for tweet in tweepy.Cursor(api.search, 'data science').items(50):
    # {Tweet id: [times seen, text]}
    if 'retweeted_status' in tweet._json:
        orig_tweet = tweet.retweeted_status
        if orig_tweet.id not in result:
            result[orig_tweet.id] = [1, orig_tweet.text]
        else:
            result[orig_tweet.id][0] += 1
print(result)


In [ ]:
# Getting URLs
for tweet in tweepy.Cursor(api.search, 'data science').items(100):
    urls = tweet.entities['urls']
    for url in urls:
        print(url['expanded_url'])

In [ ]:
# Getting WL tweets and timestamps

for tweet in tweepy.Cursor(api.search, 'geocode:40.42,-86.9,20km').items(100):
    print(str(tweet.created_at) + '\t' + tweet.text)
    

In [ ]:
# Word used (non-streaming)
words_used = {}
for tweet in tweepy.Cursor(api.search, 'geocode:40.42,-86.9,20km').items(100):
    text = tweet.text.split()
    for word in text:
        if word not in words_used:
            words_used[word] = 1
        else:
            words_used[word] += 1

words_tuple = words_used.items()
words_tuple

In [ ]:
# Filtering streaming by location
class StreamListener(tweepy.StreamListener):
    def on_status(self, tweet):
        print(tweet.author.screen_name + "\t" + tweet.text)

    def on_error(self, status_code):
        print( 'Error: ' + repr(status_code))
        return False

l = StreamListener()
streamer = tweepy.Stream(auth=auth, listener=l)
streamer.filter(locations = [-87,40,-86,41])

In [9]:
# Getting followers and followers of followers

my_id = 'jdfoote'

follower_dict = {my_id:[]}

i = 1
# This takes 15 minutes for each 15 followers.
# Let me know if you found a faster way!
for follower in api.followers_ids(my_id)[:15]: # Filtered to first 15 for testing
    print('Getting user {}'.format(i))
    i += 1
    follower_dict[my_id].append(follower)
    try:
        follower_dict[follower] = api.followers_ids(follower)
    except tweepy.error.TweepError:
        continue

TweepError: Failed to send request: ('Connection aborted.', OSError("(104, 'ECONNRESET')"))

In [ ]:
my_followers = set(follower_dict['jdfoote'])

max_matches = (0, None)
for person, followers in follower_dict.items():
    followers = set(followers)
    matches = len(my_followers & followers)
    if matches > max_matches[0]:
        max_matches = (matches, person)

In [ ]:
non_matches = 0
for person, followers in follower_dict.items():
    followers = set(followers)
    matches = len(my_followers & followers)
    if matches == 0:
        non_matches += 1
        print(person)
print(non_matches)